In [ ]:
from   astropy import time
import json
from   plotsettings import *
from   standard_libraries import *
import requests
#from   scipy import interpolate

In [ ]:
def api(method, endpoint, data=None):
    """API request"""

    headers = {'Authorization': f'token {myToken}'}
    print(headers)
    response = requests.request(method, endpoint, json=data, headers=headers)
    print(f'HTTP code: {response.status_code}, {response.reason}')
    return response

def get_groups(source):
    """Get the groups a source belongs to"""

    response = api('GET',
                   f'https://fritz.science/api/sources/{source}'
                   )

    if response.status_code == 200:
        groups = response.json()['data']['groups']
        return groups
    
    else:
        print(f'HTTP code: {response.status_code}, {response.reason}')
        return None


In [ ]:
root = os.path.expanduser('~') + '/NOT/'
#root = '/Volumes/Home/steve/Downloads/'
#root = '/Volumes/Work/pynot/'
path = root# + 'raw/'

In [ ]:
os.chdir(path)

In [ ]:
!ls -lrt *ascii

In [ ]:
object_name = 'ZTF23aaxuvkn'
date        = '20231220'

In [ ]:
files = sorted(glob.glob('{name}_NOT_ALFOSC_{date}_combine_*ascii'.format(name=object_name, date=date)))
#files = sorted(glob.glob('ZTF22abnfjsm_NOT_ALFOSC_20231215_combine_objid_1_*ascii'))
files

In [ ]:
plt.figure(figsize=(9*np.sqrt(2),9))
ax=plt.subplot(111)

ymax = []

for file in files:
	print(file)
	_data = ascii.read(file, names=('WAVE', 'FLUX', 'FLUX_ERR'), header_start=0)
	ax.plot(_data['WAVE'], _data['FLUX'])

	print(np.nanmedian(_data['FLUX'][_data['WAVE'] > 4000.]))
	ymax.append(np.nanmedian(_data['FLUX'][_data['WAVE'] > 4000.]))

ax.set_xlim(3000, 10000)
ax.set_ylim(0, 10*np.nanmedian(ymax));

In [ ]:
_data.meta['comments']

In [ ]:
# Push to Fritz

In [ ]:
# Add your token

In [ ]:
myToken = 'YOUR_TOKEN'

In [ ]:
present_groups = [x['id'] for x in get_groups(object_name)]
present_groups

In [ ]:
# Choose the file index that you want to upload

In [ ]:
idx_file = 2

In [ ]:
flag_submit = False if present_groups == None else True	

In [ ]:
#file = open(files[idx_file], mode='r')

In [ ]:
# User IDs
# Jesper: 23, Steve 39, Sean 1297, Priscila 1305

In [ ]:
for x in _data.meta['comments'][:10]:
    print(x)

In [ ]:
data = {
"observed_by":   [23, 39],                                                                   # Jesper, Steve
"group_ids":     list([int(x) for x in np.union1d(present_groups, [41, 87, 1])]),            # Share with existing groups AND RCF, Jesper's group and Sitewise
"filename":      files[idx_file],
"reduced_by":    [39],                                                                       # Steve
"instrument_id": 26,                                                                         # ALFOSC
"observed_at":   [x.replace('DATE-OBS: ', '') for x in _data.meta['comments'] if 'DATE-OBS' in x][0],
"obj_id":        [x.replace('OBJECT: ', '') for x in _data.meta['comments'] if 'OBJECT' in x][0],
"ascii":         open(files[idx_file]).read(),
"wave_column": 0,
"flux_column": 1,
"fluxerr_column": 2
}

In [ ]:
[key + ': ' + str(data[key]) for key in data.keys() if key != 'ascii']

In [ ]:
data

In [ ]:
if flag_submit:
    response = api('POST', 'https://fritz.science/api/spectrum/ascii', data=data)
    response